# 빠른 시작

AlphaGenome 빠른 시작 가이드에 오신 것을 환영합니다! 이 튜토리얼 노트북의 목표는
모델을 사용하고 예측을 수행하는 방법을 빠르게 익힐 수 있도록 돕는 것입니다.

## 라이브러리 임포트

In [ ]:
from alphagenome.data import gene_annotation
from alphagenome.data import genome
from alphagenome.data import transcript as transcript_utils
from alphagenome.interpretation import ism
from alphagenome.models import dna_client
from alphagenome.models import variant_scorers
from alphagenome.visualization import plot_components
import matplotlib.pyplot as plt
import pandas as pd

## DNA 시퀀스에 대한 출력 예측

AlphaGenome은 DNA 시퀀스로부터 예측을 수행하는 모델입니다. 모델을 로드해보겠습니다:

In [ ]:
API_KEY = "<your_api_key_here>"
dna_model = dna_client.create(API_KEY)

모델은 다음과 같은 [OutputType](https://www.alphagenomedocs.com/exploring_model_metadata.html)들에 대한 예측을 수행할 수 있습니다:

In [ ]:
[output.name for output in dna_client.OutputType]

AlphaGenome은 OutputType당 여러 '트랙'을 예측하여 다양한 조직과 세포 유형을 다룹니다. 
하지만 몇 개의 관심 있는 subset만 추린다면 효율적으로 예측할 수 있습니다.

다음은 길이 1MB의 DNA 시퀀스에 대해 폐 조직(`ontology_terms`로 지정)에 해당하는 
트랙 하위 집합에서 DNase-seq 예측(`OutputType`으로 지정)을 수행하는 방법입니다:

*참고: 조직과 세포 유형에 대한 일관되고 널리 인정받는 분류를 제공하기 위해 
UBERON(해부학용) 및 세포 온톨로지(CL) 등의 표준화된 생물학적 소스의 온톨로지 용어를 사용합니다.*

In [ ]:
output = dna_model.predict_sequence(
    sequence='GATTACA'.center(
        dna_client.SEQUENCE_LENGTH_1MB, 'N'
    ),  # 유효한 시퀀스 길이로 패딩
    requested_outputs=[dna_client.OutputType.DNASE],
    ontology_terms=['UBERON:0002048'],  # 폐
)

`output` 객체는 요청된 모든 다양한 OutputType에 대한 예측을 포함합니다   
(이 경우 `DNASE` OutputType만). 유전체 트랙에 대한 예측은 `TrackData` 객체 내부에 저장됩니다:

In [ ]:
dnase = output.dnase
type(dnase)

`TrackData` 객체는 다음과 같은 구성요소들을 가지고 있습니다:

<a href="https://services.google.com/fh/files/misc/trackdata.png"><img src="https://services.google.com/fh/files/misc/trackdata.png" alt="trackdata" border="0" height=500></a>

`(sequence_length, num_tracks)` 형태의 예측값은 `.values`에 저장됩니다:

In [ ]:
print(dnase.values.shape)

dnase.values

그리고 각 트랙을 설명하는 해당 메타데이터는 `.metadata`에 저장됩니다:

In [ ]:
dnase.metadata

이 경우 출력 트랙이 하나뿐이므로 트랙 메타데이터는 1개 행만 반환합니다.

트랙 메타데이터는 여러 조직이나 세포 유형에 대한 예측을 요청할 때와  
스트랜드를 구분하는 어세이(CAGE 및 RNA-seq와 같이 두 DNA 스트랜드에 대해 별도의 판독값을 가지는 어세이)를  
다룰 때 특히 유용합니다:

In [ ]:
sequence = 'GATTACA'.center(dna_client.SEQUENCE_LENGTH_1MB, 'N')
output = dna_model.predict_sequence(
    sequence=sequence,  # Use the padded sequence
    requested_outputs=[
        dna_client.OutputType.CAGE,
        dna_client.OutputType.DNASE,
    ],
    ontology_terms=[
        'UBERON:0002048',  # Lung.
        'UBERON:0000955',  # Brain.
    ],
)

print(f'DNASE predictions shape: {output.dnase.values.shape}')
print(f'CAGE predictions shape: {output.cage.values.shape}')

In [ ]:
# Create a fake interval for visualization
fake_interval = genome.Interval(
    chromosome='sequence',  # Arbitrary name
    start=1,
    end=len(sequence),  # Your sequence length
    strand='+'
)

# Set the interval on your output data
output.dnase.interval = fake_interval
output.cage.interval = fake_interval

# Now visualize
plot_components.plot(
    components=[
        plot_components.Tracks(output.dnase),
        plot_components.Tracks(output.cage),
    ]
)

In [ ]:
# Create a fake interval for visualization
fake_interval = genome.Interval(
    chromosome='sequence',
    start=1,
    end=len(sequence),
    strand='+'
)

# Create new TrackData objects with the interval
dnase_with_interval = output.TrackData(
    values=output.dnase.values,
    interval=fake_interval,
    resolution=output.dnase.resolution,
    ontology_terms=output.dnase.ontology_terms
)

cage_with_interval = output.TrackData(
    values=output.cage.values,
    interval=fake_interval,
    resolution=output.cage.resolution,
    ontology_terms=output.cage.ontology_terms
)

# Now visualize
plot_components.plot(
    components=[
        plot_components.Tracks(dnase_with_interval),
        plot_components.Tracks(cage_with_interval),
    ]
)

이 예시에서는 2개의 어세이와 2개의 온톨로지 용어에 대한 예측을 동시에 요청했다는 점에 주목하세요.

CAGE 트랙 메타데이터는 예측된 4개 트랙 각각의 스트랜드와 조직을 설명합니다 
(DNA 스트랜드당 2개):

In [ ]:
output.cage.metadata

Output type과 Output shape에 대한 자세한 정보는 
[출력 메타데이터 문서](https://www.alphagenomedocs.com/exploring_model_metadata.html)를 참조하세요.   
조직 이름(예: 'brain' -> 'UBERON:0000955')과 온톨로지 용어 간의 매핑은 
이 [튜토리얼](tissue_ontology_mapping.ipynb)을 참조하세요.

## 유전체 구간에 대한 출력 예측 (참조 유전체)

편의를 위해 **유전체 구간**으로 지정된 인간 참조 유전체 시퀀스에 대해서도 직접 예측할 수 있습니다.  
예를 들어, 약물 대사에 관련된 효소를 인코딩하고 주로 간에서 발현되는 *CYP2B6* 유전자 주변의 
19번 염색체 1MB 영역에서 '우측 간엽' 조직에 대한 RNA-seq를 예측해보겠습니다.

먼저 GENCODE에 의해 주석이 달린 유전자 및 전사체 위치가 포함된 GTF 파일을 로드합니다 
(GTF 형식에 대한 자세한 정보는 [여기](https://www.gencodegenes.org/pages/data_format.html)를 참조):

In [ ]:
# The GTF file contains information on the location of all trancripts.
# Note that we use genome assembly hg38 for human.
gtf = pd.read_feather(
    'https://storage.googleapis.com/alphagenome/reference/gencode/'
    'hg38/gencode.v46.annotation.gtf.gz.feather'
)

# Set up transcript extractors using the information in the GTF file.
# Mane select transcripts consists of of one curated transcript per locus.
gtf_transcripts = gene_annotation.filter_protein_coding(gtf)
gtf_transcripts = gene_annotation.filter_to_mane_select_transcript(gtf_transcripts)
transcript_extractor = transcript_utils.TranscriptExtractor(gtf_transcripts)

그 다음 `gene_symbol`(HGNC 명명 규칙) 또는 ENSEMBL `gene_id`를 전달하여 
유전자의 위치를 `genome.Interval` 객체로 가져옵니다:

In [ ]:
interval = gene_annotation.get_gene_interval(gtf, gene_symbol='CYP2B6')
interval

모델과 호환되는 길이로 크기를 조정할 수 있습니다:

In [ ]:
interval = interval.resize(dna_client.SEQUENCE_LENGTH_1MB)

`.resize()` 메서드는 원래 중심을 기준으로 확장(또는 축소)하여 구간을 지정된 너비로 조정합니다.   
`dna_model.predict_interval()`은 이 크기 조정을 패딩 토큰이 아닌 실제 유전체 시퀀스의 확장으로 해석한다는 점에 주의하세요.

In [ ]:
interval.width

`resize`와 같은 더 많은 유용한 명령어는 
[필수 명령어 문서](https://www.alphagenomedocs.com/colabs/essential_commands.html)를 참조하세요.

AlphaGenome은 다음과 같은 입력 시퀀스 길이를 지원합니다:

In [ ]:
dna_client.SUPPORTED_SEQUENCE_LENGTHS.keys()

이제 구간을 사용하여 예측할 수 있습니다:

In [ ]:
output = dna_model.predict_interval(
    interval=interval,
    requested_outputs=[dna_client.OutputType.RNA_SEQ],
    ontology_terms=['UBERON:0001114'],
)  # Right liver lobe.

output.rna_seq.values.shape

일반적으로 주어진 온톨로지 용어에 대해 여러 트랙을 가질 수 있습니다. 
이 경우 "우측 간엽" 조직에 대해 3개의 RNA-seq 트랙을 가지고 있습니다.

이러한 예측을 시각화해보겠습니다. 예측된 트랙과 함께 유전자 전사체를 시각화하는 것이 유용하므로 
여기서 추출해보겠습니다:

In [ ]:
transcripts = transcript_extractor.extract(interval)
print(f'Extracted {len(transcripts)} transcripts in this interval.')

모델 API에서 반환되는 `TrackData` 및 기타 객체와 잘 통합되는 
[시각화 기초 가이드](https://www.alphagenomedocs.com/visualization_library_basics.html)도 제공합니다.

In [ ]:
plot_components.plot(
    components=[
        plot_components.TranscriptAnnotation(transcripts),
        plot_components.Tracks(output.rna_seq),
    ],
    interval=output.rna_seq.interval,
)

plt.show()

이 플롯은 예측된 3개의 RNA-seq 트랙을 시각화하고 1MB 영역에서 유전자당 MANE select 전사체의 위치도 표시합니다.

구간 크기를 조정하여 플롯의 중간 부분을 확대할 수 있습니다:

In [ ]:
plot_components.plot(
    components=[
        plot_components.TranscriptAnnotation(
            transcripts, fig_height=0.1
        ),
        plot_components.Tracks(output.rna_seq),
    ],
    interval=output.rna_seq.interval.resize(2**15),
)

plt.show()

여기서 예측된 RNA-seq 값이 엑손의 위치와 잘 정렬되어 있고, 예측이 스트랜드별로 구분되어 있음을 볼 수 있습니다.  
유전자가 위치한 양의 스트랜드에서 예측된 값이 훨씬 높습니다. 전사체의 화살표가 왼쪽에서 오른쪽으로 가므로  
*CYP2B6* 유전자가 양의 스트랜드에 있다는 것을 알 수 있습니다.

시각화 라이브러리에 대한 자세한 내용은 
[시각화 기초 가이드](https://www.alphagenomedocs.com/visualization_library_basics.html)와 
[라이브러리 문서](https://www.alphagenomedocs.com/api/visualization.html)를 참조하세요.

## 변이 효과 예측

참조(REF)와 대안(ALT) 대립유전자 시퀀스에 대한 예측을 수행하여 특정 출력 유형과 조직에 대한 변이의 효과를 예측할 수 있습니다.

`genome.Variant` 객체를 정의하여 변이를 지정합니다. 아래의 특정 변이는 대장 조직에서 유전자 발현에 영향을 미치는 것으로 알려진 변이입니다:

In [ ]:
variant = genome.Variant(
    chromosome='chr22',
    position=36201698,
    reference_bases='A',  # Can differ from the true reference genome base.
    alternate_bases='C',
)

다음으로 REF 및 ALT 예측을 수행할 구간을 정의합니다. `genome.Variant`에서 `genome.Interval`을 얻는 빠른 방법은  
`.reference_interval`을 호출하는 것이며, 이를 모델 호환 시퀀스 길이로 크기 조정할 수 있습니다:

In [ ]:
interval = variant.reference_interval.resize(dna_client.SEQUENCE_LENGTH_1MB)

그 다음 `predict_variant`를 사용하여 "횡행결장" 조직(`UBERON:0001157`)에 대한 구간의 REF 및 ALT RNA-seq 예측을 얻습니다:

In [ ]:
variant_output = dna_model.predict_variant(
    interval=interval,
    variant=variant,
    requested_outputs=[dna_client.OutputType.RNA_SEQ],
    ontology_terms=['UBERON:0001157'],
)  # Colon - Transverse.

예측된 REF 및 ALT 값을 단일 플롯으로 그리고 영향을 받은 유전자를 확대하여 유전자 발현에 대한 변이의 효과를 더 잘 시각화할 수 있습니다:

In [ ]:
transcripts = transcript_extractor.extract(interval)

plot_components.plot(
    [
        plot_components.TranscriptAnnotation(transcripts),
        plot_components.OverlaidTracks(
            tdata={
                'REF': variant_output.reference.rna_seq,
                'ALT': variant_output.alternate.rna_seq,
            },
            colors={'REF': 'dimgrey', 'ALT': 'red'},
        ),
    ],
    interval=variant_output.reference.rna_seq.interval.resize(2**15),
    # Annotate the location of the variant as a vertical line.
    annotations=[plot_components.VariantAnnotation([variant], alpha=0.8)],
)
plt.show()

ALT 대립유전자(위치 36201698의 염기 'C')가 음의 스트랜드에 있는 *APOL4* 유전자에서 낮은 발현과 엑손 스키핑 이벤트 모두와 연관되어 있음을 볼 수 있습니다.   
양의 DNA 스트랜드에서 매우 미미한 예측 발현량을 보여주는 최상단 라인 플롯은 무시할 수 있습니다(y축 스케일 확인).   
y축 한계를 조정하는 것이 가능합니다.   
[시각화 기초](https://www.alphagenomedocs.com/visualization_library_basics.html#visualization-library-basics)와 
[라이브러리 문서](https://www.alphagenomedocs.com/api/visualization.html)를 참조하세요.

## 유전 변이의 효과 점수화

유전 변이의 효과를 점수화하는 것은 REF와 ALT 시퀀스에 대한 예측을 수행하고 트랙 신호를 집계하는 것을 포함합니다.   
이는 집계를 위해 특정 `variant_scorer` 설정을 사용하는 `score_variant`에서 구현됩니다.  

모든 출력 유형을 다루는 권장 변이 점수화 설정 세트를   
딕셔너리(`variant_scorers.RECOMMENDED_VARIANT_SCORERS`)로 제공하며,  
이는 도메인별 작업에서의 성능을 평가한 것입니다. 자세한 정보는   
[변이 점수화 문서](https://www.alphagenomedocs.com/variant_scoring.html)를 참조하세요.   
다음은 간단한 데모입니다:

In [ ]:
variant_scorer = variant_scorers.RECOMMENDED_VARIANT_SCORERS['RNA_SEQ']

variant_scores = dna_model.score_variant(
    interval=interval, variant=variant, variant_scorers=[variant_scorer]
)

반환된 `variant_scores`는 스코어를 1개만 지정했기 때문에 길이가 1인 리스트입니다:

In [ ]:
len(variant_scores)

변이당 실제 점수는 `AnnData` 형식이며, 이는 행과 열에 대한 추가 정보로 데이터(수치 점수)에 주석을 다는 방법입니다.

In [ ]:
variant_scores = variant_scores[0]
variant_scores

In [ ]:
variant_scores

`AnnData` 객체는 다음과 같은 구성요소들을 가지고 있습니다:

<a href="https://services.google.com/fh/files/misc/anndata.png"><img src="https://services.google.com/fh/files/misc/anndata.png" alt="anndata" border="0" height=500></a>

구간의 37개 유전자 각각과 667개의 `RNA_SEQ` 트랙 각각에 대한 변이 효과 점수가 있습니다:

In [ ]:
variant_scores.X.shape

`.obs`를 사용하여 37개 유전자에 대한 정보에 액세스할 수 있습니다. 다음은 처음 5개 유전자입니다:

In [ ]:
variant_scores.obs.head()

유전자별이 아닌 변이 스코어(즉, `variant_scorers.CenterMaskScorer`)를 사용하는 경우,  
`variant_scores.X`는 `(1, 667)` 형태를 가지며 이 시나리오에서는 유전자의 개념이 없기 때문에  
유전자 메타데이터를 사용할 수 없습니다.  

각 트랙의 설명은 `.var`를 사용하여 액세스됩니다 (이는 출력 메타데이터와 동일한 데이터프레임이지만   
편의를 위해 변이 점수와 함께 포함됩니다):

In [ ]:
variant_scores.var

몇 가지 유용한 추가 메타데이터는 `.uns`에서 찾을 수 있습니다:

In [ ]:
print(f'Interval: {variant_scores.uns["interval"]}')
print(f'Variant: {variant_scores.uns["variant"]}')
print(f'Variant scorer: {variant_scores.uns["variant_scorer"]}')

`tidy_scores`를 사용하여 `AnnData` 객체를 평면화하여 변이 점수와 상호작용하는 것을 권장합니다.  
이는 (변이, 유전자, 스코어러, 온톨로지)의 각 조합에 대해 단일 점수가 있는 각 행이 포함된 데이터프레임을 생성합니다.  
유전자별 스코어러의 경우 유전자의 스트랜드와 일치하지 않는 스트랜드 트랙을 선택적으로 제외합니다.  

`raw_score` 열은 `variant_scores.X`에 저장된 것과 동일한 값을 포함합니다.  
`quantile_score` 열은 일반적인 변이의 배경 세트에 대한 점수 분포에서 `raw_score`의 순위이며,  
분위수 확률로 표현됩니다. 이를 통해 서로 다른 척도에서 점수를 산출하는 변이 점수화 전략 간의 직접적인 비교가 가능합니다.  
자세한 내용은 [FAQ](https://www.alphagenomedocs.com/faqs.html#what-is-the-difference-between-a-quantile-score-and-raw-score)를 참조하세요.

In [ ]:
variant_scorers.tidy_scores([variant_scores], match_gene_strand=True)

## *in silico* 돌연변이 유발을 통한 중요 영역 강조

DNA 시퀀스에서 최종 변이 예측에 기능적으로 중요한 영역을 강조하기 위해, 특정 구간의  
모든 가능한 단일 뉴클레오타이드 변이를 시뮬레이션하고 변이의 효과를 점수화하는  
**in silico 돌연변이 유발**(ISM) 분석을 수행할 수 있습니다.

다음은 이 과정의 시각적 개요입니다:

<a href="https://services.google.com/fh/files/misc/ism_green_v2.png"><img src="https://services.google.com/fh/files/misc/ism_green_v2.png" alt="ISM" border="0" height=500></a>

체계적으로 돌연변이를 일으키고자 하는 상대적으로 작은 DNA 영역인 `ism_interval`을 정의합니다.  
또한 각 변이에 대한 예측을 수행할 때 모델이 사용할 맥락 구간인 `sequence_interval`도 정의합니다.

In [ ]:
# 16KB DNA sequence to use as context when making predictions.
sequence_interval = genome.Interval('chr20', 3_753_000, 3_753_400)
sequence_interval = sequence_interval.resize(dna_client.SEQUENCE_LENGTH_16KB)

# Mutate all bases in the central 256-base region of the sequence_interval.
ism_interval = sequence_interval.resize(256)

다음으로 각 ISM 변이를 점수화하는 데 사용할 scorer를 정의합니다.  
여기서는 예측된 `DNASE` 값에 대한 중앙 center mask scorer를 사용하며,  
이는 500bp 주변에서 DNA 접근성에 대한 각 변이의 효과를 점수화합니다.   
변이 점수화에 대한 자세한 정보는 [변이 점수화 문서](https://www.alphagenomedocs.com/variant_scoring.html)를 참조하세요.

In [ ]:
dnase_variant_scorer = variant_scorers.CenterMaskScorer(
    requested_output=dna_client.OutputType.DNASE,
    width=501,
    aggregation_type=variant_scorers.AggregationType.DIFF_MEAN,
)

마지막으로 `score_variants`(복수형 s에 주목)를 사용하여 모든 변이를 점수화할 수 있습니다.

이 작업은 상당히 비용이 많이 듭니다. 속도상의 이유로 가능한 경우 
맥락적 `sequence_interval`에 대해서는 더 짧은 입력 시퀀스를, 
돌연변이시킬 `ism_interval` 영역은 더 좁게 사용하는 것을 권장합니다.

In [ ]:
variant_scores = dna_model.score_ism_variants(
    interval=sequence_interval,
    ism_interval=ism_interval,
    variant_scorers=[dnase_variant_scorer],
)

768개의 변이(256개 위치 * 위치당 3개 대안 염기)를 점수화했으므로 
반환된 `variant_scores`의 길이는 768입니다:

In [ ]:
len(variant_scores)

각 변이는 `(1, 305)` 형태의 점수를 가지며, 이는 유전자 중심 스코어러를 사용하지 않고 
305개의 `DNASE` 트랙이 있다는 사실을 반영합니다:

In [ ]:
# Index into first variant and first scorer.
variant_scores[0][0].X.shape

예측에서 어떤 위치가 가장 영향력이 있는지 이해하기 위해 시퀀스 로고를 사용하여 
이러한 점수를 시각화할 수 있습니다. 이를 위해서는 점수를 변이당 단일 스칼라 값으로 요약해야 합니다.

예를 들어, 널리 사용되는 실험 모델인 K562 세포주에 대한 DNASE 점수만 추출해보겠습니다. 
또는 단일 스칼라 값을 얻기 위해 여러 조직에 걸쳐 평균을 낼 수도 있습니다.

In [ ]:
def extract_k562(adata):
  values = adata.X[:, adata.var['ontology_curie'] == 'EFO:0002067']
  assert values.size == 1
  return values.flatten()[0]


ism_result = ism.ism_matrix(
    [extract_k562(x[0]) for x in variant_scores],
    variants=[v[0].uns['variant'] for v in variant_scores],
)

`ism_result`의 형태는 4개의 DNA 염기 각각에 대해 위치당 1개의 점수를 가지므로 `(256, 4)`입니다.

이 경우 `ism.ism_matrix()`에 대한 호출에서 `multiply_by_sequence` 인수가 'True'로 설정되었으므로, 
출력 배열은 참조 시퀀스에 해당하는 염기에 대해서만 0이 아닌 값을 포함합니다.

In [ ]:
ism_result.shape

마지막으로 기여도 점수를 시퀀스 로고로 그립니다:

In [ ]:
plot_components.plot(
    [
        plot_components.SeqLogo(
            scores=ism_result,
            scores_interval=ism_interval,
            ylabel='ISM K562 DNase',
        )
    ],
    interval=ism_interval,
    fig_width=35,
)

plt.show()

이 플롯은 위치 ~225에서 ~240 사이의 시퀀스가 K562 세포에서 예측된 근처 DNAse에 가장 강한 영향을 미친다는 것을 보여줍니다.

이러한 기여도 점수는 다양한 모달리티와 세포 유형에 중요한 모티프를 체계적으로 발견하고, 
그 모티프에 결합하는 전사 인자를 찾고, 유전체 전반에 걸쳐 모티프 인스턴스를 매핑하는 데 사용될 수 있습니다. 
다음은 이를 위해 사용할 수 있는 몇 가지 도구입니다:
- [tfmodisco-lite](https://github.com/jmschrei/tfmodisco-lite/)
- [tangermeme](https://github.com/jmschrei/tangermeme) 
- [tomtom](https://meme-suite.org/meme/tools/tomtom)

## Mouse 예측 수행

지금까지 이 노트북은 인간(`Organism.HOMO_SAPIENS`)에 대한 예측에 중점을 두었습니다. 
마우스에 대한 예측을 생성하려면 대신 organism을 `Organism.MUS_MUSCULUS`로 지정하세요. 
지원되는 온톨로지 용어는 종에 따라 다르다는 점에 주의하세요.

다음 예시는 마우스 예측을 위해 `predict_sequence`를 호출하는 방법을 보여줍니다:

In [ ]:
output = dna_model.predict_sequence(
    sequence='GATTACA'.center(
        dna_client.SEQUENCE_LENGTH_1MB, 'N'
    ),  # Pad to valid sequence length.
    organism=dna_client.Organism.MUS_MUSCULUS,
    requested_outputs=[dna_client.OutputType.DNASE],
    ontology_terms=['UBERON:0002048'],  # Lung.
)

그리고 다음은 mouse 유전체 구간에 대해 `predict_interval`을 호출하는 예시입니다:

In [ ]:
interval = genome.Interval('chr1', 3_000_000, 3_000_001).resize(
    dna_client.SEQUENCE_LENGTH_1MB
)

output = dna_model.predict_interval(
    interval=interval,
    organism=dna_client.Organism.MUS_MUSCULUS,
    requested_outputs=[dna_client.OutputType.RNA_SEQ],
    ontology_terms=['UBERON:0002048'],  # Lung.
)

output.rna_seq.values.shape

## Conclusion

That's it for the quick start guide. To dive in further, check out our
[other tutorials](https://www.alphagenomedocs.com/tutorials/index.html).